In [57]:
%matplotlib inline 
import cv2
from __future__ import division
from pylab import *
import skimage as ski
from skimage import data, io, filters, exposure
from skimage.filters import rank
from skimage import img_as_float, img_as_ubyte
from skimage.morphology import disk
import skimage.morphology as mp
from skimage import util
from skimage.color import rgb2hsv, hsv2rgb, rgb2gray
from skimage.filters.edges import convolve
from matplotlib import pylab as plt
import numpy as np
from numpy import array
from IPython.display import display
from ipywidgets import interact, interactive, fixed
from ipywidgets import *
from ipykernel.pylab.backend_inline import flush_figures

def nasyc(img):
    [[cell for cell in row] for row in img][2]=255
    return img

def plot_hist(img):
    img = img_as_ubyte(img)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    histo, x = np.histogram(img, range(0, 256), density=True)
    plot(histo)
    xlim(1, 255)
    ylim(0, 0.02)

def main():
    kernel = np.ones((4,4),np.uint8)
    tablice=["1", "cytryna", "merc", "test", "tunel"]
    for i, tablica in enumerate(tablice):
        tablice[i] = tablica + ".png"
    
    for i in range(1,10):
        tablice.append("sam" + str(i) + ".png")
        
    for tablica in tablice:
        print(tablica)
        zdjecie=cv2.imread("Samochody/" + tablica)
        
        zdjecie=cv2.cvtColor(zdjecie,cv2.COLOR_BGR2HSV)
        #cv2.imshow("begin", nasyc(zdjecie))
        
        #blurred = cv2.GaussianBlur(zdjecie, (3, 3), 0)
        
        low=np.array([0,0,150])
        high=np.array([255,30,250])
        #nas = np.array([[cell[1] for cell in row] for row in zdjecie])
        
        #maska = np.uint8((nas < 50) * 255)
        
        
        low2=np.array([100,60,20])
        high2=np.array([130,255,200])
        
        maska=cv2.inRange(zdjecie,low,high)
        maska2=cv2.inRange(zdjecie,low2,high2)
        
        
        res=cv2.bitwise_and(zdjecie,zdjecie,mask=maska)
        res=cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
        
        
        res=cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
        
        #res = cv2.adaptiveThreshold(res ,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,7)
        #res = (res - np.nanmean(res)) / np.nanstd(res)
        #res[res > 0.3] = 1
        #res[res < 0.3] = 0
        #res *= 255
        
        #res=cv2.Canny(np.uint8(zdjecie),len(res),len(res[0]))
        #res = filters.sobel(res)
        
        res2=cv2.bitwise_and(zdjecie,zdjecie,mask=maska2)
        res2=cv2.cvtColor(res2,cv2.COLOR_HSV2BGR)
        
        #res = mp.erosion(res)
        
        res=cv2.morphologyEx(res,cv2.MORPH_OPEN, kernel)
        #res=cv2.morphologyEx(res,cv2.MORPH_CLOSE,kernel)
        #res = 255 - res
        #res = cv2.morphologyEx(res,cv2.MORPH_OPEN,kernel)
        
        # Initiate FAST object with default values
        fast = cv2.FastFeatureDetector()

        # find and draw the keypoints
    
        
        
        cv2.imwrite("Samochody_filtr/B" + tablica, res )
        cv2.imwrite("Samochody_filtr/N" + tablica, res2 )
        #res,thresh=cv2.threshold(res,128,255,cv2.THRESH_BINARY)
        #thresh=res
        
        #cv2.imshow("thresh000",thresh)
        #thresh=cv2.Canny(thresh,len(thresh),len(thresh[0]))

    
        #cv2.imshow("zdjecie",zdjecie)
        #cv2.imshow("maska",maskafinal)
        #cv2.imshow("maska2",maska2)
        #cv2.imshow("res",res)
        #cv2.imshow("thresh",thresh)
        
        #cv2.waitKey(0)
    
    
if __name__=="__main__":
    main()

1.png
cytryna.png
merc.png
test.png
tunel.png
sam1.png
sam2.png
sam3.png
sam4.png
sam5.png
sam6.png
sam7.png
sam8.png
sam9.png


In [55]:
# USAGE
# python detect_shapes.py --image shapes_and_colors.png

# import the necessary packages
from pyimagesearch.shapedetector import ShapeDetector
import argparse
import imutils
import cv2

# construct the argument parse and parse the arguments

# load the image and resize it to a smaller factor so that
# the shapes can be approximated better
image = cv2.imread("Samochody_filtr/Bsam1.png")
#imagge = cv2.resize(image, (960, 540)) 
resized = imutils.resize(image, width=300)
ratio = image.shape[0] / float(resized.shape[0])
# convert the resized image to grayscale, blur it slightly,
# and threshold it
gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
#gray = gray[30:30, 270:270]
#blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)[1]
thresh = gray
# find contours in the thresholded image and initialize the
# shape detector
#cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
#	cv2.CHAIN_APPROX_SIMPLE)
cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#print(cnts[-1])
#cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = cnts[1]
sd = ShapeDetector()

cv2.namedWindow("Image", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Image",3,cv2.WINDOW_FULLSCREEN)
# loop over the contours
for c in cnts:
    # compute the center of the contour, then detect the name of the
    # shape using only the contour        
    M = cv2.moments(c)
    if M["m00"] == 0.0:            
        continue
    cX = int((M["m10"] / M["m00"]) * ratio)
    cY = int((M["m01"] / M["m00"]) * ratio)
    shape = sd.detect(c)
    if shape != "rectangle":
        continue
    # multiplyly the contour (x, y)-coordinates by the resize ratio,
    # then draw the contours and the name of the shape on the image
    c = c.astype("float")
    c *= ratio
    c = c.astype("int")
    cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
    cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
        0.5, (255, 255, 255), 2)
    
    # show the output image

cv2.imshow("Image", image)
cv2.waitKey(0)

-1